# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [3]:
import tensorflow as tf
import numpy as np
device = '/device:gpu:1'
%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [4]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [5]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [6]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0,seed=42)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device('/device:gpu:0'):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()


(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [8]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0,seed=42)
        self.conv1 = tf.keras.layers.Conv2D(filters=channel_1,kernel_size=(5,5), activation='relu',padding='same',
                                   kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(filters=channel_2,kernel_size=(3,3), activation='relu',padding='same',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc3(x)
        

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return x

In [9]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [10]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False,print_every = 100):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            # train_loss.reset_states()
            # train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        # val_loss.reset_states()
                        # val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [11]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0009889602661133, Accuracy: 10.9375, Val Loss: 2.950167417526245, Val Accuracy: 12.700000762939453
Iteration 100, Epoch 1, Loss: 2.2542262077331543, Accuracy: 28.434404373168945, Val Loss: 2.4361326694488525, Val Accuracy: 24.75
Iteration 200, Epoch 1, Loss: 2.084911823272705, Accuracy: 32.22170639038086, Val Loss: 2.2400596141815186, Val Accuracy: 29.69999885559082
Iteration 300, Epoch 1, Loss: 2.0046002864837646, Accuracy: 34.105064392089844, Val Loss: 2.1558661460876465, Val Accuracy: 31.450000762939453
Iteration 400, Epoch 1, Loss: 1.9358911514282227, Accuracy: 35.719295501708984, Val Loss: 2.070573091506958, Val Accuracy: 33.5
Iteration 500, Epoch 1, Loss: 1.8904615640640259, Accuracy: 36.72343063354492, Val Loss: 2.000894546508789, Val Accuracy: 35.266666412353516
Iteration 600, Epoch 1, Loss: 1.8595675230026245, Accuracy: 37.7261848449707, Val Loss: 1.9540117979049683, Val Accuracy: 36.4428596496582
Iteration 700, Epoch 1, Loss: 1.8340221643447876, 

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [12]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1,channel_2,num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,nesterov=True, momentum=0.9)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn,print_every=10)

Iteration 0, Epoch 1, Loss: 3.01434326171875, Accuracy: 20.3125, Val Loss: 5.303304672241211, Val Accuracy: 13.0
Iteration 10, Epoch 1, Loss: 3.1446175575256348, Accuracy: 18.323863983154297, Val Loss: 3.7688117027282715, Val Accuracy: 16.69999885559082
Iteration 20, Epoch 1, Loss: 2.6898348331451416, Accuracy: 20.684524536132812, Val Loss: 3.2262210845947266, Val Accuracy: 19.133333206176758
Iteration 30, Epoch 1, Loss: 2.4940707683563232, Accuracy: 22.782258987426758, Val Loss: 2.926913261413574, Val Accuracy: 21.875
Iteration 40, Epoch 1, Loss: 2.36868953704834, Accuracy: 25.22865867614746, Val Loss: 2.7297816276550293, Val Accuracy: 24.1200008392334
Iteration 50, Epoch 1, Loss: 2.2780258655548096, Accuracy: 26.13357925415039, Val Loss: 2.5925514698028564, Val Accuracy: 25.433334350585938
Iteration 60, Epoch 1, Loss: 2.204115390777588, Accuracy: 27.561477661132812, Val Loss: 2.4865174293518066, Val Accuracy: 26.914287567138672
Iteration 70, Epoch 1, Loss: 2.1504828929901123, Accurac

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [13]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0,seed=42)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0009889602661133, Accuracy: 10.9375, Val Loss: 2.950167417526245, Val Accuracy: 12.700000762939453
Iteration 100, Epoch 1, Loss: 2.2542262077331543, Accuracy: 28.434404373168945, Val Loss: 2.4361326694488525, Val Accuracy: 24.75
Iteration 200, Epoch 1, Loss: 2.084911823272705, Accuracy: 32.22170639038086, Val Loss: 2.2400596141815186, Val Accuracy: 29.69999885559082
Iteration 300, Epoch 1, Loss: 2.0046002864837646, Accuracy: 34.105064392089844, Val Loss: 2.1558661460876465, Val Accuracy: 31.450000762939453
Iteration 400, Epoch 1, Loss: 1.9358911514282227, Accuracy: 35.719295501708984, Val Loss: 2.070573091506958, Val Accuracy: 33.5
Iteration 500, Epoch 1, Loss: 1.8904615640640259, Accuracy: 36.72343063354492, Val Loss: 2.000894546508789, Val Accuracy: 35.266666412353516
Iteration 600, Epoch 1, Loss: 1.8595675230026245, Accuracy: 37.7261848449707, Val Loss: 1.9540117979049683, Val Accuracy: 36.4428596496582
Iteration 700, Epoch 1, Loss: 1.8340221643447876, 

Альтернативный менее гибкий способ обучения:

In [14]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)



313/313 [==============================] - 2s 5ms/step - loss: 1.6812 - sparse_categorical_accuracy: 0.4151


[1.6812174320220947, 0.41510000824928284]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [15]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0,seed=42)
    layers = [
        tf.keras.layers.Conv2D(filters=channel_1,kernel_size=(5,5),input_shape = input_shape, activation='relu',padding='same',
                                   kernel_initializer=initializer),
        tf.keras.layers.Conv2D(filters=channel_2,kernel_size=(3,3), activation='relu',padding='same',
                                   kernel_initializer=initializer),                           
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,nesterov=True, momentum=0.9)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.01434326171875, Accuracy: 20.3125, Val Loss: 2.7612833976745605, Val Accuracy: 10.40000057220459
Iteration 100, Epoch 1, Loss: 2.034350633621216, Accuracy: 28.96039581298828, Val Loss: 2.2920219898223877, Val Accuracy: 23.450000762939453
Iteration 200, Epoch 1, Loss: 1.898222804069519, Accuracy: 33.861942291259766, Val Loss: 2.092512369155884, Val Accuracy: 29.766666412353516
Iteration 300, Epoch 1, Loss: 1.8226604461669922, Accuracy: 36.290489196777344, Val Loss: 1.9790370464324951, Val Accuracy: 33.64999771118164
Iteration 400, Epoch 1, Loss: 1.758641242980957, Accuracy: 38.524784088134766, Val Loss: 1.894837737083435, Val Accuracy: 36.5
Iteration 500, Epoch 1, Loss: 1.7105251550674438, Accuracy: 40.094810485839844, Val Loss: 1.835629940032959, Val Accuracy: 38.21666717529297
Iteration 600, Epoch 1, Loss: 1.679160714149475, Accuracy: 41.11896896362305, Val Loss: 1.7863787412643433, Val Accuracy: 39.75714111328125
Iteration 700, Epoch 1, Loss: 1.650217175

In [16]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)


313/313 [==============================] - 1s 3ms/step - loss: 1.3836 - sparse_categorical_accuracy: 0.5045


[1.3835784196853638, 0.5044999718666077]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [17]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [18]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1505627632141113, Accuracy: 6.25, Val Loss: 3.0024468898773193, Val Accuracy: 14.399999618530273
Iteration 100, Epoch 1, Loss: 2.211041212081909, Accuracy: 28.589109420776367, Val Loss: 2.446993589401245, Val Accuracy: 25.35000228881836
Iteration 200, Epoch 1, Loss: 2.060417413711548, Accuracy: 32.190608978271484, Val Loss: 2.2494893074035645, Val Accuracy: 29.600000381469727
Iteration 300, Epoch 1, Loss: 1.9887984991073608, Accuracy: 33.98567199707031, Val Loss: 2.150414228439331, Val Accuracy: 31.775001525878906
Iteration 400, Epoch 1, Loss: 1.9216675758361816, Accuracy: 35.90632629394531, Val Loss: 2.067791700363159, Val Accuracy: 33.79999923706055
Iteration 500, Epoch 1, Loss: 1.877763271331787, Accuracy: 37.03530502319336, Val Loss: 1.999660611152649, Val Accuracy: 35.18333435058594
Iteration 600, Epoch 1, Loss: 1.8484934568405151, Accuracy: 37.91597366333008, Val Loss: 1.9542477130889893, Val Accuracy: 36.28571319580078
Iteration 700, Epoch 1, Loss: 

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [19]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channel_1, channel_2, num_classes_relu, num_classes_softmax = 32, 64, 100, 10
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Сверточный слой 1: 32 фильтра, MaxPool
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (3, 3), activation='relu', padding='same', kernel_initializer=initializer)
        self.mp1 = tf.keras.layers.MaxPooling2D((2, 2))

        # Сверточный слой 2: 64 фильтра, MaxPool
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), activation='relu', padding='same', kernel_initializer=initializer)
        self.mp2 = tf.keras.layers.MaxPooling2D((2, 2))
        self.d1 = tf.keras.layers.Dropout(0.25)

        # Используем flatten для сглаживания и Dropout для того, чтобы избежать переобучения
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes_relu, activation='relu')
        self.d2 = tf.keras.layers.Dropout(0.25)
        self.fc2 = tf.keras.layers.Dense(num_classes_softmax, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.mp1(x)
        x = self.conv2(x)
        x = self.mp2(x)
        x = self.d1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.d2(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

In [20]:
model = model_init_fn()
model.compile(optimizer=optimizer_init_fn(),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=70, epochs=10, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

Epoch 1/10
700/700 [==============================] - 12s 15ms/step - loss: 1.3798 - sparse_categorical_accuracy: 0.5241 - val_loss: 1.2042 - val_sparse_categorical_accuracy: 0.5880
Epoch 2/10
700/700 [==============================] - 10s 14ms/step - loss: 1.0542 - sparse_categorical_accuracy: 0.6317 - val_loss: 0.9811 - val_sparse_categorical_accuracy: 0.6550
Epoch 3/10
700/700 [==============================] - 11s 16ms/step - loss: 0.9117 - sparse_categorical_accuracy: 0.6859 - val_loss: 0.9896 - val_sparse_categorical_accuracy: 0.6590
Epoch 4/10
700/700 [==============================] - 12s 17ms/step - loss: 0.8045 - sparse_categorical_accuracy: 0.7210 - val_loss: 0.9360 - val_sparse_categorical_accuracy: 0.6890
Epoch 5/10
700/700 [==============================] - 11s 16ms/step - loss: 0.7180 - sparse_categorical_accuracy: 0.7523 - val_loss: 0.9609 - val_sparse_categorical_accuracy: 0.6880
Epoch 6/10
700/700 [==============================] - 11s 15ms/step - loss: 0.6516 - spars

[1.0856927633285522, 0.6948000192642212]

In [21]:
model.summary()

Model: "custom_conv_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Bat  multiple                  12        
 chNormalization)                                                
                                                                 
 conv2d_9 (Conv2D)           multiple                  896       
                                                                 
 batch_normalization_3 (Bat  multiple                  128       
 chNormalization)                                                
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        multiple                  0         
                                                 